In [22]:
import torch
from data import MePicsSet
import morph, utils, my_types
from torch.utils.data import DataLoader
from learn import test
import time

import matplotlib.pyplot as plt

import morph
import utils
from IPython.display import Video

In [18]:
april_dataset = MePicsSet()
april_loader = DataLoader(april_dataset, batch_size=8, shuffle=False)

In [19]:
img_names = [sample.get_name() for sample in april_dataset.samples]
print(img_names)

['me_2002.jpg', 'me_2003.jpeg', 'me_2011_0.jpg', 'me_2011_1.jpg', 'me_2012.jpg', 'me_2013.jpg', 'me_2014_2.jpg', 'me_2015.jpg', 'me_2016.jpg', 'me_2017_0.jpg', 'me_2017_1.jpg', 'me_2018.jpg', 'me_2019_0.jpg', 'me_2019_1.jpg', 'me_2020_0.jpg', 'me_2020_1.jpg']


In [20]:
model3 = torch.load("output/model3.pt", map_location="cpu")

In [21]:
april_imgs, april_preds = test(april_loader, model3)

100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


In [ ]:
for i, p in zip(april_imgs, april_preds):
    show_keypoints(image=i, truth_points=None, pred_points=p)

In [ ]:
morph_imgs = []
morph_pts = []
corners = np.array([[0, 0], [0, 499], [499, 0], [499, 499]])
for i in range(len(april_dataset)):

    im = utils.to_img_arr(april_dataset.get_original_img(i, cropped=True))
    #     print(im.shape)
    morph_imgs.append(im)
    # TODO add corners
    pred_pts = utils.to_points(april_preds[i])
    pts = np.vstack((pred_pts, corners))
    #     print(len(pts))
    morph_pts.append(pts)

In [ ]:
# plt.imshow(img)
middle, mid_pts, del_tri = morph.compute_middle_object(
    morph_imgs[0], morph_imgs[1], morph_pts[0], morph_pts[1], alpha=0.5
)
plt.imshow(middle)
warped1 = morph.warp_img(morph_imgs[0], morph_imgs[1], morph_pts[0], morph_pts[1], alpha=0.5
)

In [ ]:
filename = OUT_DIR / f"morph_{time.time()}.mp4"

me1, me2 = morph_imgs[0], morph_imgs[1]
pts1, pts2 = morph_pts[0], morph_pts[1]
# morph.compute_middle_object(me1, me2, pts1, pts2, alpha=0.5)
# morph.compute_morph_video(me1, me2, pts1, pts2, filename)

In [ ]:
filename = OUT_DIR / "morph_all.mp4"
morph.make_giant_video(morph_imgs, morph_pts, fps=25, filename=filename)


In [ ]:
Video("output/morph_all.mp4")